# Docker Entrypoint with Scripts

### Introduction

So in the last lesson, we saw how Entrypoint is a Docker command that is run at runtime, and that we can pass additional arguments to at runtime.  In this lesson, we'll see how we can use Entrypoint to set up our web applications.

### An Entrypoint Usecase

Ok, so remember that what's unique about entrypoint is that, unlike CMD, it cannot be overridden.   It is always run when booting up a container.  So when working with a web application, a typical usecase would be some initial setup.  

For example, we may need to create and seed our database before running our flask application.  So let's say that we want to **an create our database 

### Setting up docker

Ok, so now let's build the docker image.  The Dockerfile is a little tricky, so we have done this for you.  The key issue is that we want multiple things to occur when we boot up our docker container (aka "at runtime").  When we create a docker container we want to:

* Create our database tables (if they do not already exist), like the `positions` and `scrapings` tables
* Seed our `positions` and `scrapings` tables if they do not already have data in them.
* Run our flask application by default.  

Ok, so to achieve this we do a couple of things:

1. Using entrypoint and command in our Dockerfile

If you look at the Dockerfile, you'll see this at the bottom.

```Dockerfile
ENTRYPOINT ["sh", "./setup.sh"]

CMD ["flask", "run", "--host=0.0.0.0"]
```

Here's how it works.  

The `Entrypoint` is always run at run time (when you boot up the container), and CMD contains the default arguments passed to what's specified in entrypoint.  So in this case, it's as if we are doing:

`sh ./setup.sh flask run --host=0.0.0.0`

This will run the `./setup.sh` and then pass `flask run --host=0.0.0.0` to that file.

What's accomplished by running `sh ./setup.sh flask run --host=0.0.0.0`?  Let's talk about this next.

2. `setup.sh` file

To understand the above command, you'll need to look at the `setup.sh` file.  If you look at the `setup.sh` file you'll see the following:

```bash
flask init-db
flask seed-db

exec "$@"
```

So this will call our `init-db` and `seed-db` functions to create and seed our tables.  The `exec "$@"` allows us to pass optional bash commands to the script.  So when we setup our Dockerfile to run the script with:

`sh ./setup.sh flask run --host=0.0.0.0`

The arguments of `flask run --host=0.0.0.0` will be passed into that `exec` line, and thus be called.

We can play around with this.  For example, if we run `sh setup.sh echo hello world`, then we will have created and seeded our tables and run displayed hello world at the end.

> Do this now.

But in the scenario we encoded, the `CMD ["flask", "run", "--host=0.0.0.0"]` says to pass `flask run --host=0.0.0.0` to our entrypoint `sh setup.sh`, so then the `setup.sh` file executes the `flask run --host=0.0.0.0` command after first creating the tables and seeding the database.

And remember we can override that default command at run time with something like:

`docker run image_name flask run --debug=True`

And that means that the setup.sh script will catch those arguments and run that instead of the original command.

* Building our image

Ok, so back to the show.  Now build the image, but do so with tagging the image with your dockerhub username first.  Here's an example, so swap our `jek2141` with your username.

```bash
docker build -t jek2141/scraper_backend .
```

So now we'll want to bootup our image locally before trying it on our ec2 instance, but doing so will be a fairly long line.  So you may want to write it out in the `ec2-setup.sh` file, and the copy and paste it into your terminal.  

Ok, so boot up your container, but make sure you pass through environmental variables `docker build -e` for all of the database environmental variables. 

If it works, you should be able to go to `localhost:5000/positions` and see the positions in the flask application.  

* Make sure that your environmental variables are properly getting passed through by passing through some incorrect information (like a wrong password) that should cause your application to break.  If this doesn't work it means you are likely reading from the .env file but not from your `docker build -e` arguments.


* One more thing

Now we're about to move onto terraform, but there is one issue with our docker image that we'll likely run into.  It's that there may be a mismatch between our laptop where we built our image on and the ec2 machine we ultimately use.  So before moving on, let's rebuild the image and tag it.  

> Just replace `jek2141` with your username.

`docker build -t jek2141/scraper_backend:amd_v2 --platform=linux/amd64/v2 .`

And now this is the image we'll ultimate want to use on our ec2 machine, so let's push it up to dockerhub.

`docker push jek2141/scraper_backend:amd_v2`

### Summary

In this lesson, we first organized our codebase so that files like `.env` and `.flaskenv` are under the correct folders.  And we then added some functions to more easily set up our database.  We confirmed these commands worked before then encapsulating this in a docker image.


We then used our Dockerfile and a `setup.sh` file to call these functions.  And we did so by running the `setup.sh` file from our dockerfile with the entrypoint command.

```dockerfile
ENTRYPOINT ["sh", "./setup.sh"]
```

And then the command function sets default arguments that will be passed to the entrypoint command when we run `docker run`.

```dockerfile
CMD ["flask", "run", "--host=0.0.0.0"]
```

When this was complete, we built our image.  We properly tagged it so that we could push it to dockerhub.  We also specified the platform to ensure it would run on our ec2 machine.

### Resources

[Terraform working with Following](https://discuss.hashicorp.com/t/template-v2-2-0-does-not-have-a-package-available-mac-m1/35099/3)